# Bidirectional Contradictions Analysis

7 cases where BOTH bidir_true AND bidir_false proofs succeeded.
These come from 2 stories only.

In [1]:
import json
import re

with open('../data/folio/original/folio-validation.json') as f:
    data = json.load(f)

contradiction_cases = [75, 76, 77, 156, 157, 158, 159]

## Story 368: Mary's School (Cases 75, 76, 77)

In [2]:
# Story 368 - shared premises
print("=" * 80)
print("STORY 368: PREMISES")
print("=" * 80)
print(data[75]['premises'])
print()

STORY 368: PREMISES
If people at Mary's school work in student jobs on campus, then they need to earn money to help pay for their college tuition.
If people at Mary's school order takeout frequently in college, then they work in student jobs on campus.
People at Mary's school order takeout frequently in college or enjoy the dining hall meals and recipes.
If people at Mary's school enjoy the dining hall meals and recipes, then they are not picky eaters.
If people at Mary's school enjoy the dining hall meals and recipes, then they spend a lot of their time eating and catching up with friends in the campus dining halls.
Hannah is at Mary's school.
Hannah works in student jobs on campus and if she needs to earn money to help pay for her college tuition, then she is neither picky nor needs to earn money to help pay for her college tuition.



In [3]:
# Story 368 - conclusions for each case
for idx in [75, 76, 77]:
    print(f"\n--- Case {idx} (GT: {data[idx]['label']}) ---")
    print(f"Conclusion: {data[idx]['conclusion']}")


--- Case 75 (GT: Uncertain) ---
Conclusion: Hannah needs to earn money to help pay for her college tuition.

--- Case 76 (GT: True) ---
Conclusion: Hannah is at Mary's school and she is not a picky eater and spends a lot of her time eating and catching up with friends in the campus dining halls.

--- Case 77 (GT: False) ---
Conclusion: Hannah is at Mary's school and she either is not a picky eater or, if she is, then she spends a lot of her time eating and catching up with friends in the campus dining halls.


## Story 435: Library / CS Department (Cases 156, 157, 158, 159)

In [4]:
# Story 435 - shared premises
print("=" * 80)
print("STORY 435: PREMISES")
print("=" * 80)
print(data[156]['premises'])
print()

STORY 435: PREMISES
None of the students who currently work in the library are from the computer science department. 
All of the students with part-time jobs offered by the university currently work in the library. 
All of the students who are taking the database course are from the computer science department. 
All of the students taking a class with Professor David are taking the database course. 
All students working in the lab are taking a class with Professor David.
James is a student working in the lab. 
James doesn't work in the lab or have a part-time job offered by the university.



In [5]:
# Story 435 - conclusions for each case
for idx in [156, 157, 158, 159]:
    print(f"\n--- Case {idx} (GT: {data[idx]['label']}) ---")
    print(f"Conclusion: {data[idx]['conclusion']}")


--- Case 156 (GT: False) ---
Conclusion: James has a part-time job offered by the university.

--- Case 157 (GT: True) ---
Conclusion: James does not have a part-time job offered by the university.

--- Case 158 (GT: True) ---
Conclusion: James either takes the database course or has a part-time job offered by the university.

--- Case 159 (GT: Uncertain) ---
Conclusion: James takes the database course.


## Lean Code Comparison

In [6]:
def get_lean_code(case_idx, condition):
    if condition == 'true':
        path = f'../results/simplelean/gpt-5_folio_bidir_true_20251224_123917/responses/case_{case_idx}.txt'
    else:
        path = f'../results/simplelean/gpt-5_folio_bidir_false_20251224_125254/responses/case_{case_idx}.txt'
    
    with open(path) as f:
        content = f.read()
    
    match = re.search(r'<lean>(.*?)</lean>', content, re.DOTALL)
    return match.group(1).strip() if match else None

In [7]:
# Case 77 comparison (most interesting - GT is False)
print("=" * 80)
print("CASE 77 - bidir_true (said True, GT=False)")
print("=" * 80)
print(get_lean_code(77, 'true'))

CASE 77 - bidir_true (said True, GT=False)
axiom obj : Type
axiom Hannah : obj

axiom AtMS : obj → Prop
axiom Work : obj → Prop
axiom Need : obj → Prop
axiom Order : obj → Prop
axiom Enjoy : obj → Prop
axiom Picky : obj → Prop
axiom Spend : obj → Prop

-- Rules from the context
axiom R1 : ∀ x : obj, AtMS x → Work x → Need x
axiom R2 : ∀ x : obj, AtMS x → Order x → Work x
axiom R3 : ∀ x : obj, AtMS x → (Order x ∨ Enjoy x)
axiom R4 : ∀ x : obj, AtMS x → Enjoy x → ¬ Picky x
axiom R5 : ∀ x : obj, AtMS x → Enjoy x → Spend x

-- Facts about Hannah
axiom T1 : AtMS Hannah
axiom T2a : Work Hannah
axiom T2b : Need Hannah → (¬ Picky Hannah ∧ ¬ Need Hannah)

theorem hannah_statement :
  AtMS Hannah ∧ (¬ Picky Hannah ∨ (Picky Hannah → Spend Hannah)) :=
by
  have hAt : AtMS Hannah := T1
  have hWork : Work Hannah := T2a
  have hNeed : Need Hannah := R1 Hannah hAt hWork
  have hNotPicky : ¬ Picky Hannah := And.left (T2b hNeed)
  exact And.intro hAt (Or.inl hNotPicky)


In [8]:
print("=" * 80)
print("CASE 77 - bidir_false (said False, GT=False) - CORRECT")
print("=" * 80)
print(get_lean_code(77, 'false'))

CASE 77 - bidir_false (said False, GT=False) - CORRECT
axiom obj : Type

axiom Hannah : obj

-- Predicates
axiom At : obj → Prop                          -- at Mary's school
axiom Work : obj → Prop                        -- works in student jobs on campus
axiom Need : obj → Prop                        -- needs to earn money to help pay for college tuition
axiom OrderTakeout : obj → Prop                -- orders takeout frequently in college
axiom EnjoyDining : obj → Prop                 -- enjoys the dining hall meals and recipes
axiom Picky : obj → Prop                       -- is a picky eater
axiom SpendDining : obj → Prop                 -- spends a lot of time eating and catching up with friends in the campus dining halls

-- Rules restricted to people at Mary's school
axiom R1 : ∀ x : obj, At x ∧ Work x → Need x
axiom R2 : ∀ x : obj, At x ∧ OrderTakeout x → Work x
axiom R3 : ∀ x : obj, At x → (OrderTakeout x ∨ EnjoyDining x)
axiom R4 : ∀ x : obj, At x ∧ EnjoyDining x → ¬Picky x
a

In [9]:
# Case 156 comparison
print("=" * 80)
print("CASE 156 - bidir_true")
print("=" * 80)
print(get_lean_code(156, 'true'))

CASE 156 - bidir_true
axiom obj : Type
axiom James : obj

axiom WorkInLibrary : obj → Prop
axiom WorkInLab : obj → Prop
axiom PTJobUniv : obj → Prop
axiom TakingDatabase : obj → Prop
axiom FromCS : obj → Prop
axiom TakingWithDavid : obj → Prop

-- None of the students who currently work in the library are from the CS department.
axiom A1 : ∀ x : obj, WorkInLibrary x → ¬ FromCS x
-- All of the students with part-time jobs offered by the university currently work in the library.
axiom A2 : ∀ x : obj, PTJobUniv x → WorkInLibrary x
-- All of the students who are taking the database course are from the CS department.
axiom A3 : ∀ x : obj, TakingDatabase x → FromCS x
-- All of the students taking a class with Professor David are taking the database course.
axiom A4 : ∀ x : obj, TakingWithDavid x → TakingDatabase x
-- All students working in the lab are taking a class with Professor David.
axiom A5 : ∀ x : obj, WorkInLab x → TakingWithDavid x

-- James is a student working in the lab.
axiom T

In [10]:
print("=" * 80)
print("CASE 156 - bidir_false")
print("=" * 80)
print(get_lean_code(156, 'false'))

CASE 156 - bidir_false
axiom Person : Type
axiom James : Person

axiom Student : Person → Prop
axiom WorkLibrary : Person → Prop
axiom FromCS : Person → Prop
axiom PartTimeUni : Person → Prop
axiom TakingDB : Person → Prop
axiom TakingWithDavid : Person → Prop
axiom WorkLab : Person → Prop

-- None of the students who currently work in the library are from the CS department.
axiom A1 : ∀ x : Person, (Student x ∧ WorkLibrary x) → ¬ FromCS x
-- All students with university part-time jobs currently work in the library.
axiom A2 : ∀ x : Person, (Student x ∧ PartTimeUni x) → WorkLibrary x
-- All students taking the database course are from the CS department.
axiom A3 : ∀ x : Person, (Student x ∧ TakingDB x) → FromCS x
-- All students taking a class with Professor David are taking the database course.
axiom A4 : ∀ x : Person, (Student x ∧ TakingWithDavid x) → TakingDB x
-- All students working in the lab are taking a class with Professor David.
axiom A5 : ∀ x : Person, (Student x ∧ WorkLab x